In [2]:
import os
import json
from typing import List, Tuple, Optional
from dotenv import load_dotenv
from hume import HumeBatchClient
from hume.models.config import ProsodyConfig
from collections import defaultdict

In [ ]:
# TODO: Hume AI; integrate into main later on 
def analyze_audio_emotions(filepath: str) -> Optional[List[Tuple[str, float]]]:
    load_dotenv()

    client_key = os.getenv('HUME_AI_CLIENT_KEY')
    client_secret = os.getenv('HUME_AI_CLIENT_SECRET')

    if not client_key or not client_secret:
        raise ValueError("HUME_AI_CLIENT_KEY and HUME_AI_CLIENT_SECRET must be set in .env file")

    try:
        # Initialize Hume client and submit job
        client = HumeBatchClient(client_key)
        config = ProsodyConfig()
        job = client.submit_job(None, [config], files=[filepath])
        
        print("Analyzing audio...")
        job.await_complete()
        job.download_predictions("predictions.json")
        print("Analysis complete. Predictions downloaded to predictions.json")

        # Process the predictions
        with open('predictions.json', 'r') as file:
            data = json.load(file)

        # Aggregate emotion scores; there has to be a simpler way to do this
        emotion_scores = defaultdict(float)
        for prediction in data[0]['results']['predictions']:
            for group in prediction['models']['prosody']['grouped_predictions']:
                for pred in group['predictions']:
                    for emotion in pred['emotions']:
                        emotion_scores[emotion['name']] += emotion['score']

        # Find the top 2 emotions
        top_2_emotions = sorted(emotion_scores.items(), key=lambda x: x[1], reverse=True)[:2]

        return top_2_emotions

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [ ]:
# Testing
audio_file = "./media/test-vids/test.mp4"  
result = analyze_audio_emotions(audio_file)

if result:
    print("Top 2 emotions overall:")
    for emotion, score in result:
        print(f"{emotion}: {score:.2f}")
else:
    print("Failed to analyze audio emotions.")